In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Chemical Index',use_remote=False)

In [ ]:
# fetch data set
df_cas = fh.get_df(os.path.join(hndl.sandbox_dir,'workdf.parquet'))


In [ ]:
# # make Open-FF's common name (FF now has IngredientCommonName)
# gb2 = df_cas.groupby('bgCAS')['IngredientName'].agg(lambda x: x.value_counts().index[0])
# gb2 = gb2.reset_index()
# gb2.columns = ['bgCAS','ingredCommonName']
# df_cas = pd.merge(df_cas,gb2,on='bgCAS',how='left')
# # df_cas.columns

The interactive table below lists all chemicals (from the Open-FF data set) published in FracFocus after curation. Some chemicals are listed that have no records in the filtered set, but are in the full, raw data. Chemical identity is established using both CASNumber and IngredientName, when possible.  See the [CAS/Ingredient Index](Open-FF_CASNumber_and_IngredientName.html) for more detail on these identities. Below the table is an explanation of this table's columns.

The table is sort-able by any column (click a second time for reverse sort) and use the **Search** bars at the bottom of each column to limit what is shown in the table. 

The following table is representative of the Std_filtered data only - there may be chemicals that only show up in disclosures that we filter. 

In [ ]:
import openFF.common.chem_list_summary as chem_sum
chem_obj =  chem_sum.ChemListSummary(df_cas)
t = chem_obj.get_display_table(colset='chem_index')

In [ ]:
scifi_df = pd.read_parquet(r"G:\My Drive\webshare\scrape_data\SciFinder_chem_pages\scifinder_df.parquet")
scifi_df['comp1'] = scifi_df['comp1'].apply(list)
scifi_df['comp2'] = scifi_df['comp2'].apply(list)
scifi_df['comps'] = scifi_df.comp1 + scifi_df.comp2
print(scifi_df.columns)
print(t.columns)
t.head()
t = t.merge(scifi_df[['bgCAS','comps','subs_class']],on='bgCAS',how='left')
t = t.drop('bgCAS',axis=1)
t = t.rename({'comps':'components','subs_class':'substance_class'},axis=1)

In [ ]:
# iShow(t.reset_index(drop=True),maxBytes=0, 
#       columnDefs=[{"width": "100px", "targets": 0},
#                   {"targets": [1,4,7,11],"visible": False }],
#       buttons=[{
#         "extend": "colvis",
#         # "collectionLayout": "fixed columns",
#         "popoverTitle": "Column visibility control"
#     }],column_filters="footer", layout={"topEnd": None})

In [ ]:
iShow(t.reset_index(drop=True),maxBytes=0, autowidth=False,lengthMenu=[5,10, 20, 50,100,500],
       search={"regex": True, "caseInsensitive": True, "search": ""},
      columnDefs=[{"width": "50px", "targets": '_all'},
                  {"targets": [1,4,7,11,12,13],"visible": False }
                 ],
      buttons=['pageLength',{
        "extend": "colvis",
        "popoverTitle": "Column visibility control"
        }],
      # , paging=True,
        # column_filters="footer",
      # layout={"topEnd": None},
      classes="display wrap compact cell-border")

**Important note about chemical names**: most chemical compounds have a number of names they are known by.  For example, the chemical with the CAS registration number 111-76-4 is listed as:

| name | source |
| --- |--- |
|ethanol, 2-butoxy-|SciFinder (primary name)|
|Ethylene glycol monobutyl ether| EPA registry name|
|Ethylene glycol monobutyl ether (2-butoxyethanol)| Elsner/Hoelzer's paper|
|2-butoxyethanol| most common name in FracFocus|
| 2-BE |informal, shorthand name|

In the table above, two names are given for each material, but many others are often used. That might be a problem if you are trying to find something by a different name.  If you are not having luck in the table above, try exploring the [synonyms index](Open-FF_Synonyms.html) for the most appropriate CAS Number.

|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*Material*| at top is the CAS registration number of the chemical. **Click on this link** to display summary of this chemical for all of FracFocus (filtered data - up to last major update). Below are two names for the material: the upper is used by SciFinder as the primary name and the lower is the most common reported name in FracFocus| 
|*ref*|Click on the "csv" link to a csv-formatted file that has all the filtered records for this chemical.  Click on "ChemID" link for the ChemID website for this chemical.  Click on "EPA: CompTox" for the summary page in that resource|
|*Molecule*| is the chemical structure (image credit: EPA CompTox and ChemID)|
|*total num records* |indicates the number of records across the FracFocus data. A given chemical may appear more than once in a disclosure.  The upper number is the number of records in the **filtered** data, and the lower value is the number of records in the **unfiltered** data.|
|*num records with mass*|indicates the number of records with mass of this chemical in the **filtered** data. Without mass, a record only records presence in the disclosure.|
|*mass of 90th percentile (lbs)*| the top 10% of records in the **filtered** data have mass equal to or greater than this value|
|*Reportable quant (lbs)*|indicates the quantity of the chemical (in pounds), above which uses must be reported if spilled, based on [40 CFR § 302.4 - Hazardous substances and reportable quantities](https://www.law.cornell.edu/cfr/text/40/302.4). This is applicable throughout most industries however fracking is in general exempt from this requirement. We show it here to illustrate what those exemptions have given to the fracking industry. There are currently a total of about 100 chemicals on that list.|
|*Chemical Functional Groups*|at the top is a coarse functional category for this material. Below is a sub-category. The source of this classification is [Elsner and Hoelzer, 2016.](https://pubs.acs.org/doi/full/10.1021/acs.est.5b02818)|
|*on external list* |indicates if the chemical is on one of the following lists.  Type the name of the list into the Search box to limit to those chemicals.<br> - **CWA**: indicates that the chemical is on the [Clean Water Act list](https://comptox.epa.gov/dashboard/chemical_lists/CWA311HS) as compiled in EPA's CompTox<br>- **AQ_CWA**: Comptox list of chemicals particularly hazardous to aquatic life<br>- **HH_CWA**: Comptox list of chemicals particularly hazardous to human health<br>- **DWSHA**: indicated that the chemical is on the EPA's [Drinking Water Safety and Health Advisory](https://comptox.epa.gov/dashboard/chemical_lists/EPADWS) list<br>- **NPDWR**: indicates that the chemical is on the [National Primary Drinking Water Regulations](https://www.epa.gov/ground-water-and-drinking-water/national-primary-drinking-water-regulations) list, compiled into a machine readable version by Angelica Fuiza and Gary Allison.<br>- **TEDX**: indicates that the chemical is on [The Endocrine Disruption Exchange](https://endocrinedisruption.org) list<br>- **prop_65**: indicates that the chemical is on [California's Proposition 65](https://oehha.ca.gov/proposition-65/proposition-65-list) list<br>- **EPA_PFAS**: indicates that the chemical is on EPA's comprehensive list of PFAS related compounds<br>- **UVCB**: indicates that the chemical is on the [TSCA list of "Unknown, Variable composition, Complex reaction products and Biological"](https://www.epa.gov/sites/default/files/2015-05/documents/uvcb.pdf) materials, meaning "these chemical substances cannot be represented by unique structures and molecular formulas."  Note that many chemicals reported in FracFocus are not on the TSCA and so will not be labeled as UVCB, even though they fit the description.<br>- **IRIS**: indicates that the chemical is included in the [IRIS program](https://comptox.epa.gov/dashboard/chemical-lists/IRIS) that "identifies and characterizes the health hazards of chemicals found in the environment."<br>- **diesel**: indicates that the chemical is [on the EPA list that](https://www.epa.gov/uic/diesel-fuels-hydraulic-fracturing-dfhf) "represent the most appropriate interpretation of the statutory term 'diesel fuels' to use for permitting diesel fuels hydraulic fracturing under the UIC Program nationwide.|
|*fingerprint*:|This is a "hazard" summary based on the [EPA's ChemInformatics Dashboard](https://www.epa.gov/chemical-research/cheminformatics). For each chemical that the EPA has published summaries, Open-FF generates a single image that indicates level of hazard for EPA's twenty categories.  See the table below or click on the specific chemical's report (its CAS number) for a different display of these results. For many chemicals reported by the EPA on that site, some of those hazard categories cannot be evaluated because the data are limited.  See EPA's disclaimer for these data below or at their site.
|*earliest date*|indicates the date of the earliest recorded use of this chemical.  By sorting on this column, you can find the chemicals that have been long used and those that have only recently been added.|
|*substance_class*|indicates the classes reported by the Chemical Abstract Services' SciFinder tool for this material.  Gives both assigned structural class and other indicators of unusual classification issues.|
|*components*|indicates the chemical components reported by the Chemical Abstract Services' SciFinder tool for this material.  These may help determine a more complete harzard profile.|



## Hazard Fingerprints
### Categories
|Row| Col 1 | Col 2| Col 3| Col 4 |Col 5|
|:---| :---: | :---: | :---: | :---: | :---:|
|**Top** |*Human Health*<br>Oral |*Human Health*<br>Inhalation |*Human Health*<br>Dermal |*Human Health*<br>Carcinogenicity|*Human Health*<br>Genotoxicity Mutagenicity|
|**Second**|*Human Health*<br>Endocrine Disruption |*Human Health*<br>Reproductive |*Human Health*<br>Developmental |*Human Health*<br>Neurotoxicity:<br> Repeat Exposure |*Human Health*<br>Neurotoxicity:<br> Single Exposure|
|**Third**|*Human Health*<br>Systemic Toxicity:<br> Repeat Exposure |*Human Health*<br>Systemic Toxicity:<br> Single Exposure  |*Human Health*<br>Skin Sensitization  |*Human Health*<br>Skin Irritation|*Human Health*<br>Eye Irritation 
|**Bottom**|*Ecotoxicity*<br>Acute Aquatic Toxicity |*Ecotoxicity*<br>Chronic Aquatic Toxicity |*Fate*<br>Persistence |*Fate*<br>Bioaccumulation |*Fate*<br>Exposure | |

### Legend
|Icon| Toxicity |
| :---: | :---: |
| <center><img src="images/ci_icons/red_skull.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Very High</H3>  |
| <center><img src="images/ci_icons/orange_exclamation.png" onerror="this.onerror=null; this.remove();" width="40"></center> |  <H3>High</H3>  |
| <center><img src="images/ci_icons/yellow-minus.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Moderate</H3>  |
| <center><img src="images/ci_icons/green-minus.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Low</H3>  |
| <center><img src="images/ci_icons/grey_question.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>Inconclusive</H3>  |
| <center><img src="images/ci_icons/grey_square.png" onerror="this.onerror=null; this.remove();" width="40"></center> | <H3>No Data</H3>  |

### EPA's disclaimer for the Hazard data
> The Hazard Comparison Dashboard is a prototype tool and a compilation of information sourced from many sites, databases and sources including U.S. Federal and state sources and international bodies that saves the user time by providing information in one location. The data are not reviewed by USEPA – the user must apply judgment in use of the information. The results do not indicate EPA’s position on the use or regulation of these chemicals. 